# Estudando sobre os métodos e dados do CNES
É uma fonte de dados incrementais, ou seja deve-se utilizar o mais atualizado possível, já que os registros estarão mais atuais.

In [1]:
from pysus.ftp.databases.cnes import CNES
cnes = CNES()

In [2]:
cnes.load()

CNES - Cadastro Nacional de Estabelecimentos de Saúde

In [3]:
cnes.groups

{'DC': 'Dados Complementares',
 'EE': 'Estabelecimento de Ensino',
 'EF': 'Estabelecimento Filantrópico',
 'EP': 'Equipes',
 'EQ': 'Equipamentos',
 'GM': 'Gestão e Metas',
 'HB': 'Habilitação',
 'IN': 'Incentivos',
 'LT': 'Leitos',
 'PF': 'Profissional',
 'RC': 'Regra Contratual',
 'SR': 'Serviço Especializado',
 'ST': 'Estabelecimentos'}

In [ ]:
group = "ST"
uf = "SP"

# Fonte de dados incrementais, portanto não há necessidade de baixar todos os dados anteriores ao mais atualizado.
# year = [i for i in range(2014, 2024+1)]
# month = [i for i in range(1, 12+1)]

year = 2024
month = 10

In [18]:
len(cnes.get_files(group, uf = uf, year = year, month = month))

0

In [ ]:
files = cnes.get_files(group, uf = uf, year = year, month = month)

In [ ]:
# Tamanho dos dados
tam = 0

for file in files:
    tam += int (file.__info__["size"])
    print(cnes.describe(file))

{'name': 'STSP2410.dbc', 'group': 'Estabelecimentos', 'uf': 'São Paulo', 'month': 'Outubro', 'year': 2024, 'size': '4.9 MB', 'last_update': '2024-11-13 01:26PM'}


In [25]:
print(f"Tamanho eh igual a {tam} B = {tam/1024} KB = {tam/(1024*1024)} MB = {tam/(1024*1024*1024)} GB.")

Tamanho eh igual a 470741030 B = 459708.037109375 KB = 448.933629989624 MB = 0.4384117480367422 GB.


## Colocando na área de stage

In [ ]:
stage_path = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/CNES/stage/parquet'

In [ ]:
parquets = cnes.download(files, local_dir = stage_path)

STSP2410.parquet: 100%|██████████| 258k/258k [02:31<00:00, 1.70kB/s]


## Analisando os atributos

In [25]:
import pandas as pd

In [26]:
dataPath = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aires/Dados/CNES/stage/parquet/STSP2410.parquet'

In [27]:
df = pd.read_parquet(dataPath)

In [29]:
print("Número de linhas: ", df.shape[0])
print("Número de colunas: ", df.shape[1])

Número de linhas:  103234
Número de colunas:  208


In [40]:
df.columns

Index(['CNES', 'CODUFMUN', 'COD_CEP', 'CPF_CNPJ', 'PF_PJ', 'NIV_DEP',
       'CNPJ_MAN', 'COD_IR', 'REGSAUDE', 'MICR_REG',
       ...
       'AP07CV02', 'AP07CV03', 'AP07CV04', 'AP07CV05', 'AP07CV06', 'AP07CV07',
       'ATEND_PR', 'DT_ATUAL', 'COMPETEN', 'NAT_JUR'],
      dtype='object', length=208)

### Inserindo no banco de dados para ver os dados melhor

In [43]:
from sqlalchemy import create_engine

# Configurações da conexão
usuario = 'postgres'
senha = '123,258D'
host = 'localhost'
porta = '5432'
banco_de_dados = 'Stage-CNES'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

In [44]:
engine = create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()

Conexão bem-sucedida!
